# Защита персональных данных клиентов

## Загрузка данных

Откроем файл с данными и изучим их, для этого подключим библиотеку `pandas` помимо неё подключим еще остальные библиотеки, которые нам пригодятся. Для того чтобы прочитать данные из датасетов воспользуемся методом `read_csv`, для получения общей информации о датасетах воспользуемся методом `info`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score

In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


По данным видно, что пропусков в столбцах нет, также сразу можно заметить, что не все столбцы имеет целочисленный тип данных, переведем все столбцы в тип *int*.

In [4]:
data = data.astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


Судя по информации, которая предоставлена в датасете, на дубликаты проверять датасет не имеет смысла, так как могут быть люди одинакового пола, возраста, с одинаковой зарплатой и одинаковым количеством членов семьи. С данными все в порядке можно переходить к разработке метода преобразования данных.

## Умножение матриц

Перед тем как переходить к разработке метода преобразования данных, ответим на следующий вопрос - **"Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?"**

Докажем алгебраически, что при умножении признаков на случайную матрицу качество модели не изменится, введем следующие обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула $MSE$ имеет следующий вид $MSE(Xw, y)$ = $ \frac{1}{n}  \sum _{i=1} ^{n}(Xw - y)^2$, умножим матрицу признаков на матрицу $P$, матрица $P$ случайная обратимая матрицу, так как матрица обратимая, то она должна быть квадратной, чтобы мы могли умножнить матрицу признаков $X$ на матрицу $P$, количество строк матрицы $P$ должно равняться количеству столбцов матрицы признаков $X$. Пусть размерность матрицы $X$ равна $(n, k)$, тогда размерность матрицы $P$ равна $(k, k)$.

Докажем, что при умножении матрицы признаков на обратимую матрицу матричное уравнение линейной регресии не изменится и будет иметь вид: 

$$
X^TXw = X^Ty
$$

Из данного уравнения выводится формула обучения, которая указана выше. Для доказательства возьмем выражение из под суммы, которая указана в формуле для $MSE$: 

$$
(Xw - y)^2
$$

Умножим в этой формуле матрицу признаков $X$ на матрицу $P$, и сразу умножим на обратную матрицу $P^{-1}$, в таком случае у нас матрица признаков не изменится, так как она в итоге умножается на единичную матрицу, а при умножении матрицы на единичную матрицу получается точно такая матрица $XE=EX=X$. В итоге у нас получится следующая формула:

$$
(XPP^{-1}w - y)^2
$$

Раскроем скобки, получим следующее выражение:

$$
(XPP^{-1}w - y)^T(XPP^{-1}w - y) = (XPP^{-1}w)^T(XPP^{-1}w) - (XPP^{-1}w)^Ty - y^{T}(XPP^{-1}w) + y^{T}y = (XPP^{-1}w)^T(XPP^{-1}w) - y^{T}(XPP^{-1}w) - (XPP^{-1}w)^Ty + y^{T}y
$$

Сделаем преобразование так, чтобы вектор $w^T$ был представлен в начале каждого произведения в уравнении. Рассмотрим следующее выражение:

$$
(XPP^{-1}w)^Ty = w^T(XPP^{-1})^Ty
$$

Проверим по размерностям матриц можем ли мы так преобразовать, размерность вектора $w=(k, 1)$, размерность вектора с целевым признаком равна $y=(n, 1)$:

$$
(XPP^{-1}w)^Ty = ((n, k)(k, k)(k, k)(k,1))^T(n, 1) = (n, 1)^T(n, 1) = (1, n)(n, 1) = (1, 1)
$$

$$
w^T(XPP^{-1})^Ty = (k, 1)^T((n, k)(k, k)(k, k))^T(n, 1) = (k, 1)^T(n, k)^T(n, 1) = (1, k)(k, n)(n, 1) = (1, n)(n, 1) = (1, 1)
$$

В обоих случаях мы получили скаляр, значит мы можем преобразовать наше произведение в уравнении по указанному выше выражению. Рассмотрим следующее выражение:

$$
(XPP^{-1}w)^T(XPP^{-1}w) = w^T(XPP^{-1})^TXPP^{-1}w
$$

Проверим по размерностям матриц можем ли мы так преобразовать:

$$
(XPP^{-1}w)^T(XPP^{-1}w) = ((n, k)(k, k)(k, k)(k, 1))^T(n, k)(k, k)(k, k)(k, 1) = (n, 1)^T(n, 1) = (1, n)(n, 1) = (1, 1)
$$

$$
w^T(XPP^{-1})^TXPP^{-1}w = (k, 1)^T((n, k)(k, k)(k, k))^T(n, k)(k, k)(k, k)(k, 1) = (1, k)(n, k)^T(n, 1) = (1, k)(k, n)(n, 1) = (1, n)(n, 1) = (1, 1)
$$

В обоих случаях мы получили скаляр, значит мы можем преобразовать наше произведение в уравнении по указанному выше выражению. Преобразуем теперь наше выражение, которое мы получили после раскрытия скобок:

$$
(XPP^{-1}w)^T(XPP^{-1}w) - y^{T}(XPP^{-1}w) - (XPP^{-1}w)^Ty + y^{T}y = w^T(XPP^{-1})^TXPP^{-1} - w^T(XPP^{-1})^Ty - (XPP^{-1}w)^Ty + y^{T}y = w^T(XPP^{-1})^TXPP^{-1} - 2w^T(XPP^{-1})^Ty + y^Ty
$$

Продифференцируем по вектору $w$ полученное выражение:

$$
\frac{d(w^T(XPP^{-1})^TXPP^{-1} - 2w^T(XPP^{-1})^Ty + (y^Ty))}{dw} 
$$

$$
2(XPP^{-1})^TXPP^{-1}w - 2(XPP^{-1})^Ty + 0 = 0
$$

$$
(XPP^{-1})^TXPP^{-1}w - (XPP^{-1})^Ty + 0 = 0
$$

$$
(XPP^{-1})^TXPP^{-1}w = (XPP^{-1})^Ty
$$

Заменим значение $PP^{-1}$ на единичную матрицу согласно выражению $PP^{-1}$=$P^{-1}P=E$:

$$
(XE)^TXEw = (XE)^Ty
$$

Как писали ранее, при умножении матрицы на единичную матрицу получается точно такая же матрица, преобразуем выражение:

$$
X^TXw = X^Ty
$$

Мы получили точно такое же уравнение линейной регресии, даже с учетом того, что мы умножили матрицу признаков $X$ на обратимую матрицу $P$ и с чего можно сделать вывод, что **при умножении признаков на обратимую матрицу качество модели Линейной регрессии не изменяется**. 

**Ответ: При умножении признаков на обратимую матрицу качество модели не изменяется.**

**Обоснование:**
При умножение матрицы опреденной размерности на обратимую квадратную матрицу у которой количество строк равно количество столбцов в первой матрице на выходе получается матрица с такой же размерностью как у первой матрицы, а **так как размерность матрицы признаков не изменяется, то и качество модели Линейной регрессии тоже не изменится**.

## Алгоритм преобразования

На основании доказательства из предыдущего пункта мы пришли к выводу, что мы при умножении матрицы признаков на обратимую матрицу качество модели **Линейной регрессии** не изменяется на основании этого мы можем использовать следующий алгоритм преобразования данных — данные, которые используют в качестве матрицы признаков, необходимо умножить на случайную обратимую квадратную матрицу с количество строк равное количество столбцов в матрице признаков, в таком случае качество нашей модели обучения не изменится, а данные преобразуются и будут защищены. Напишем функцию, которая будет преобразовывать наши данные. Для проверки получилась у нас обратимая матрица или нет, будет проверять, что определить у полученной случайной матрицы не равен нулю, так как определение обратимую матрицы следующее **матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель не равен нулю**.

In [5]:
def transform_features(features):
    """
    Функция для преобразования признаков
    """
    matrix_random = np.random.normal(size=(features.shape[1], features.shape[1]))
    if np.linalg.det(matrix_random) != 0: #проверка матрицы на обратимость через определитель
        return features @ matrix_random
    else:
        return features

Функцию написали, теперь проверим изменится ли качество модели после преобразования данных в признаках.

## Проверка алгоритма

Для начала напишем функцию, через которую будем обучать модель до и после преобразования данных, качество моделей будем проверять по метрике *R2*:

In [6]:
def learn_model(features_train, target_train, features_valid, target_valid):
    """
    Функция обучает модель линейной регрессии и проверять качество модели по метрике R2
    """
    model_linear_reg = linear_model.LinearRegression()
    model_linear_reg.fit(features_train, target_train)
    predictions = model_linear_reg.predict(features_valid)
    print("R2", r2_score(target_valid, predictions))

Функцию для обучения модели написали, напишем теперь функцию, которая будет делить данные сначала на обучающую и валидационную выборку, затем делить на признаки и целевой признак:

In [7]:
def get_data_model(data):
    """
    Функция делит данные по выборкам, в выборках разделяет данные по признака
    """
    train, valid = train_test_split(data, test_size=0.25, random_state=12345)
    print(train.shape)
    print(valid.shape)
    
    train_features = train.drop('Страховые выплаты', axis=1)
    train_target = train['Страховые выплаты']

    valid_features = valid.drop('Страховые выплаты', axis=1)
    valid_target = valid['Страховые выплаты']
    
    return train_features, train_target, valid_features, valid_target

Функцию написали, вызовем её для разделение данных по признакам:

In [8]:
data_train_features, data_train_target, data_valid_features, data_valid_target = get_data_model(data)

(3750, 5)
(1250, 5)


Как видно по размерности данные разделились корректно. Обучим теперь модель **Линейной регрессии** до преобразования данных с помощью нашей ранее написанной функции *learn_model()*

In [9]:
learn_model(data_train_features, data_train_target, data_valid_features, data_valid_target)

R2 0.4352275684083322


Качество модели до преобразования по метрике *R2* равно **0.43** преобразуем теперь наши данные, умножив матрицу признаков на обратимую матрицу, затем снова разделим данные по признакам:

In [10]:
data[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']] = \
transform_features(data[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']])
data_train_features, data_train_target, data_valid_features, data_valid_target = get_data_model(data)

(3750, 5)
(1250, 5)


Как видно после умножения на обратимую матрицу размерность матрицы признаков в обучающей и валидационной выборке не изменилась. Обучим снова модель **Линейной регрессии** и проверим изменилось ли её качество после преобразования данных:

In [11]:
learn_model(data_train_features, data_train_target, data_valid_features, data_valid_target)

R2 0.43522756840833665


Качество модели после преообразования данных по метрике *R2* равно **0.43**, такое же значение как и до преобразования данных, что говорит о том, что **качество модели не изменилось**. Если брать полностью значение по метрике *R2*, то можно заметить, что последние цифры немного отличаются, но это разница слишком мала, что ей можно пренебречь.

## Общий вывод

В рамках данной задачи нужно было подобрать алгоритм преобразования данных для их защиты так, чтобы при обучении качество модели **Линейной регрессии** не изменилось. Было доказано математически, что при умножении матрицы признаков на обратимую матрицу качество модели не изменяется, в следствии чего был выбран следующий алгоритм преобразования данных — данные, которые используют в качестве матрицы признаков, необходимо умножить на случайную обратимую квадратную матрицу с количество строк равное количество столбцов в матрице признаков, в таком случае качество нашей модели обучения не изменится, а данные преобразуются и будут защищены.

Для проверки данного алгоритма была обучена модель **Линейной регрессии** до и после преобразования данных с помощью алгоритма, в обоих случаях значение по метрике *R2* было равно **0.43**, что подтверждает, что указанный выше алгоритм подходит для преобразования данных.